# Exploring how the daily price profile varies

In [26]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

### Set charging times
Charging time 1: 02:00 - 06:00

Discharging time 1: 07:00 - 11:00

Charging time 2: 12:00 - 16:00

Discharging time 2: 16:00 - 22:00

In [27]:
price_df = pd.read_csv('../data/clean_data/wholesale_price_2022.csv', index_col=0)
price_df = price_df[['date', 'datetime', 'hour', 'gb_price_eur', 'PUN', 'NORD', 'CNOR', 'CSUD', 'SUD', 'SARD', 'SICI']]
price_df.head()

,date,datetime,hour,gb_price_eur,PUN,NORD,CNOR,CSUD,SUD,SARD,SICI
0,2022-01-01,2022-01-01 00:00:00,1,89.24,170.28,170.28,170.28,170.28,170.28,170.28,170.28
1,2022-01-01,2022-01-01 01:00:00,2,89.24,155.72,155.72,155.72,155.72,155.72,155.72,155.72
2,2022-01-01,2022-01-01 02:00:00,3,108.52,147.09,147.09,147.09,147.09,147.09,147.09,147.09
3,2022-01-01,2022-01-01 03:00:00,4,57.12,91.00,91.00,91.00,91.00,91.00,91.00,91.00
4,2022-01-01,2022-01-01 04:00:00,5,83.29,104.00,104.00,104.00,104.00,104.00,104.00,104.00


In [28]:
def hour_to_period(hour):
    if 3 <= hour <= 6: return 'charge_1'
    elif 8 <= hour <= 11: return 'discharge_1'
    elif 13 <= hour <= 16: return 'charge_2'
    elif 19 <= hour <= 22: return 'discharge_2'
    else: return 'no_flow'

price_df['flow'] = price_df.hour.apply(hour_to_period)
price_df.head()

,date,datetime,hour,gb_price_eur,PUN,NORD,CNOR,CSUD,SUD,SARD,SICI,flow
0,2022-01-01,2022-01-01 00:00:00,1,89.24,170.28,170.28,170.28,170.28,170.28,170.28,170.28,no_flow
1,2022-01-01,2022-01-01 01:00:00,2,89.24,155.72,155.72,155.72,155.72,155.72,155.72,155.72,no_flow
2,2022-01-01,2022-01-01 02:00:00,3,108.52,147.09,147.09,147.09,147.09,147.09,147.09,147.09,charge_1
3,2022-01-01,2022-01-01 03:00:00,4,57.12,91.00,91.00,91.00,91.00,91.00,91.00,91.00,charge_1
4,2022-01-01,2022-01-01 04:00:00,5,83.29,104.00,104.00,104.00,104.00,104.00,104.00,104.00,charge_1


### Splitting into charging and discharging

In [29]:
charging_1 = price_df.loc[price_df.flow == 'charge_1']
discharging_1 = price_df.loc[price_df.flow == 'discharge_1']
charging_2 = price_df.loc[price_df.flow == 'charge_2']
discharging_2 = price_df.loc[price_df.flow == 'discharge_2']
charging_1.head()

,date,datetime,hour,gb_price_eur,PUN,NORD,CNOR,CSUD,SUD,SARD,SICI,flow
2,2022-01-01,2022-01-01 02:00:00,3,108.52,147.09,147.09,147.09,147.09,147.09,147.09,147.09,charge_1
3,2022-01-01,2022-01-01 03:00:00,4,57.12,91.00,91.00,91.00,91.00,91.00,91.00,91.00,charge_1
4,2022-01-01,2022-01-01 04:00:00,5,83.29,104.00,104.00,104.00,104.00,104.00,104.00,104.00,charge_1
5,2022-01-01,2022-01-01 05:00:00,6,0.00,140.60,140.60,140.60,140.60,140.60,140.60,140.60,charge_1
26,2022-01-02,2022-01-02 02:00:00,3,226.08,140.60,140.60,140.60,140.60,140.60,140.60,140.60,charge_1


In [30]:
charging_1_max = charging_1.groupby(['date', 'flow'])['PUN'].max().reset_index()
discharging_1_min = discharging_1.groupby(['date', 'flow'])['PUN'].min().reset_index()
charging_2_max = charging_2.groupby(['date', 'flow'])['PUN'].max().reset_index()
discharging_2_min = discharging_2.groupby(['date', 'flow'])['PUN'].min().reset_index()
charging_1_max.head()

,date,flow,PUN
0,2022-01-01,charge_1,147.09000
1,2022-01-02,charge_1,141.93000
2,2022-01-03,charge_1,85.00000
3,2022-01-04,charge_1,120.00000
4,2022-01-05,charge_1,94.79055


In [31]:
cycle_1 = pd.merge(charging_1_max, discharging_1_min, on='date', how='inner', suffixes=('_charge_max', '_discharge_min'))
cycle_2 = pd.merge(charging_2_max, discharging_2_min, on='date', how='inner', suffixes=('_charge_max', '_discharge_min'))
cycle_1

,date,flow_charge_max,PUN_charge_max,flow_discharge_min,PUN_discharge_min
0,2022-01-01,charge_1,147.09000,discharge_1,67.99000
1,2022-01-02,charge_1,141.93000,discharge_1,152.59000
2,2022-01-03,charge_1,85.00000,discharge_1,202.03000
3,2022-01-04,charge_1,120.00000,discharge_1,182.64315
4,2022-01-05,charge_1,94.79055,discharge_1,215.98000
...,...,...,...,...,...
360,2022-12-27,charge_1,85.00000,discharge_1,217.88285
361,2022-12-28,charge_1,69.06998,discharge_1,202.44000
362,2022-12-29,charge_1,186.00000,discharge_1,207.82000
363,2022-12-30,charge_1,130.00000,discharge_1,199.01000


In [32]:
def charge_more_than_discharge(charge_max, discharge_min):
    if discharge_min > charge_max: return 0
    else: return 1

cycle_1['bad_profile'] = cycle_1.apply(lambda x: charge_more_than_discharge(x.PUN_charge_max, x.PUN_discharge_min), axis=1)
cycle_2['bad_profile'] = cycle_2.apply(lambda x: charge_more_than_discharge(x.PUN_charge_max, x.PUN_discharge_min), axis=1)
cycle_1.head()

,date,flow_charge_max,PUN_charge_max,flow_discharge_min,PUN_discharge_min,bad_profile
0,2022-01-01,charge_1,147.09000,discharge_1,67.99000,1
1,2022-01-02,charge_1,141.93000,discharge_1,152.59000,0
2,2022-01-03,charge_1,85.00000,discharge_1,202.03000,0
3,2022-01-04,charge_1,120.00000,discharge_1,182.64315,0
4,2022-01-05,charge_1,94.79055,discharge_1,215.98000,0


In [33]:
charging_cost = price_df.loc[(price_df.flow == 'charge_1') | (price_df.flow == 'charge_2')].PUN.sum()
discharging_income = price_df.loc[(price_df.flow == 'discharge_1') | (price_df.flow == 'discharge_2')].PUN.sum()
revenue = discharging_income - charging_cost
print(f'2022 4 hour 2 cycle revenue: {round(revenue, 2)} €/MW.')

2022 4 hour 2 cycle revenue: 205741.93 €/MW.


In [34]:
(cycle_1.PUN_discharge_min - cycle_1.PUN_charge_max).sum() + (cycle_2.PUN_discharge_min - cycle_2.PUN_charge_max).sum()

19417.519940000002

In [35]:
good_profiles_df = pd.merge(cycle_1, cycle_2, on='date', how='inner', suffixes=('_1', '_2'))
no_good_days = len(good_profiles_df.loc[(good_profiles_df.bad_profile_1 == 0) & (good_profiles_df.bad_profile_2 == 0)])
print(f'Days with a good profile on both cycles: {no_good_days}')

Days with a good profile on both cycles: 187


In [36]:
bad_profiles_df = pd.merge(cycle_1, cycle_2, on='date', how='inner', suffixes=('_1', '_2'))
no_bad_days = len(bad_profiles_df.loc[(bad_profiles_df.bad_profile_1 == 1) & (bad_profiles_df.bad_profile_2 == 1)])
print(f'Days with a bad profile on both cycles: {no_bad_days}')

Days with a bad profile on both cycles: 2


In [37]:
bad_profiles_df.loc[(bad_profiles_df.bad_profile_1 == 1) & (bad_profiles_df.bad_profile_2 == 1)]

,date,flow_charge_max_1,PUN_charge_max_1,flow_discharge_min_1,PUN_discharge_min_1,bad_profile_1,flow_charge_max_2,PUN_charge_max_2,flow_discharge_min_2,PUN_discharge_min_2,bad_profile_2
8,2022-01-09,charge_1,197.61,discharge_1,187.66,1,charge_2,223.35,discharge_2,222.80000,1
148,2022-05-29,charge_1,191.32,discharge_1,182.95,1,charge_2,185.96,discharge_2,179.55811,1


### Plot charging periods

In [38]:
cycle_1_count = 365 - cycle_1.bad_profile.sum()
cycle_2_count = 365 - cycle_2.bad_profile.sum()

fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    subplot_titles=['Cycle 1', 'Cycle 2'],
    specs=[[{'secondary_y': True}], [{'secondary_y': True}]]
)

fig.add_trace(go.Scatter(
    x=cycle_1.date,
    y=cycle_1.PUN_charge_max,
    line=dict(color='crimson', width=1.5),
    name='PUN night max'
), secondary_y=True, row=1, col=1)
fig.add_trace(go.Scatter(
    x=cycle_1.date,
    y=cycle_1.PUN_discharge_min,
    line=dict(color='green', width=1.5),
    name='PUN morning min'
), secondary_y=True, row=1, col=1)
fig.add_trace(go.Bar(
    x=cycle_1.date,
    y=cycle_1.bad_profile,
    marker_color='lightgrey',
    name='Bad profile 1'
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=cycle_2.date,
    y=cycle_2.PUN_charge_max,
    line=dict(color='indigo', width=1.5),
    name='PUN afternoon max'
), secondary_y=True, row=2, col=1)
fig.add_trace(go.Scatter(
    x=cycle_2.date,
    y=cycle_2.PUN_discharge_min,
    line=dict(color='mediumvioletred', width=1.5),
    name='PUN evening min'
), secondary_y=True, row=2, col=1)
fig.add_trace(go.Bar(
    x=cycle_2.date,
    y=cycle_2.bad_profile,
    marker_color='lightgrey',
    name='Bad profile 2'
), row=2, col=1)
fig.add_annotation(
    text=f'Cycle 1 good days: <br> {cycle_1_count}/365', 
    font=dict(size=14),
    align='center',
    showarrow=False,
    xref='paper',
    yref='paper',
    x=1.18,
    y=0.5,
    bordercolor='black',
    borderwidth=2,
    )
fig.add_annotation(
    text=f'Cycle 2 good days: <br> {cycle_2_count}/365', 
    font=dict(size=14),
    align='center',
    showarrow=False,
    xref='paper',
    yref='paper',
    x=1.18,
    y=0,
    bordercolor='black',
    borderwidth=2,
    )

fig.update_yaxes(visible=False, secondary_y=False)
fig.update_yaxes(title='€/MWh', secondary_y=True)
fig.update_layout(
    title='Italian MGP 4 hour 2 cycle analysis',
    template='plotly_white',
    width=1200,
    height=600,
)

fig.write_image('../figures/italy-4hr-profile-analysis.jpeg', scale=10, engine='orca')
fig.write_html('../figures/italy-4hr-profile-analysis.html')
fig

### Changing winter charging times

In [39]:
price_df = pd.read_csv('../data/clean_data/wholesale_price_2022.csv', index_col=0)
price_df = price_df[['date', 'datetime', 'hour', 'gb_price_eur', 'PUN', 'NORD', 'CNOR', 'CSUD', 'SUD', 'SARD', 'SICI']]
price_df.date = pd.to_datetime(price_df.date)
price_df['month'] = price_df.date.dt.month
price_winter = price_df.loc[(price_df.month <= 1) | (price_df.month >= 11)].copy()
price_summer = price_df.loc[(price_df.month > 1) & (price_df.month < 11)].copy()
price_summer.head()

,date,datetime,hour,gb_price_eur,PUN,NORD,CNOR,CSUD,SUD,SARD,SICI,month
744,2022-02-01,2022-02-01 00:00:00,1,-10.92,207.88800,208.40,208.40,208.40000,208.40000,208.40000,202.05000,2
745,2022-02-01,2022-02-01 01:00:00,2,35.99,192.56455,194.87,194.87,189.87000,189.87000,189.87000,181.92000,2
746,2022-02-01,2022-02-01 02:00:00,3,35.99,189.21848,194.16,194.16,179.26000,179.26000,179.26000,179.26000,2
747,2022-02-01,2022-02-01 03:00:00,4,-1.92,182.13464,186.10,186.10,174.04118,174.04118,174.04118,174.04118,2
748,2022-02-01,2022-02-01 04:00:00,5,12.00,180.11599,185.52,185.52,169.00000,169.00000,169.00000,169.00000,2


### Set winter charging times
Charging time 1: 02:00 - 06:00

Discharging time 1: 07:00 - 11:00

Charging time 2: 11:00 - 15:00

Discharging time 2: 16:00 - 20:00

In [40]:
def hour_to_period_summer(hour):
    if 3 <= hour <= 6: return 'charge_1'
    elif 8 <= hour <= 11: return 'discharge_1'
    elif 13 <= hour <= 16: return 'charge_2'
    elif 19 <= hour <= 22: return 'discharge_2'
    else: return 'no_flow'

price_summer['flow'] = price_summer.hour.apply(hour_to_period_summer)
price_summer.head()

,date,datetime,hour,gb_price_eur,PUN,NORD,CNOR,CSUD,SUD,SARD,SICI,month,flow
744,2022-02-01,2022-02-01 00:00:00,1,-10.92,207.88800,208.40,208.40,208.40000,208.40000,208.40000,202.05000,2,no_flow
745,2022-02-01,2022-02-01 01:00:00,2,35.99,192.56455,194.87,194.87,189.87000,189.87000,189.87000,181.92000,2,no_flow
746,2022-02-01,2022-02-01 02:00:00,3,35.99,189.21848,194.16,194.16,179.26000,179.26000,179.26000,179.26000,2,charge_1
747,2022-02-01,2022-02-01 03:00:00,4,-1.92,182.13464,186.10,186.10,174.04118,174.04118,174.04118,174.04118,2,charge_1
748,2022-02-01,2022-02-01 04:00:00,5,12.00,180.11599,185.52,185.52,169.00000,169.00000,169.00000,169.00000,2,charge_1


In [41]:
def hour_to_period_winter(hour):
    if 3 <= hour <= 6: return 'charge_1'
    elif 8 <= hour <= 11: return 'discharge_1'
    elif 12 <= hour <= 15: return 'charge_2'
    elif 17 <= hour <= 20: return 'discharge_2'
    else: return 'no_flow'

price_winter['flow'] = price_winter.hour.apply(hour_to_period_winter)
price_winter.head()

,date,datetime,hour,gb_price_eur,PUN,NORD,CNOR,CSUD,SUD,SARD,SICI,month,flow
0,2022-01-01,2022-01-01 00:00:00,1,89.24,170.28,170.28,170.28,170.28,170.28,170.28,170.28,1,no_flow
1,2022-01-01,2022-01-01 01:00:00,2,89.24,155.72,155.72,155.72,155.72,155.72,155.72,155.72,1,no_flow
2,2022-01-01,2022-01-01 02:00:00,3,108.52,147.09,147.09,147.09,147.09,147.09,147.09,147.09,1,charge_1
3,2022-01-01,2022-01-01 03:00:00,4,57.12,91.00,91.00,91.00,91.00,91.00,91.00,91.00,1,charge_1
4,2022-01-01,2022-01-01 04:00:00,5,83.29,104.00,104.00,104.00,104.00,104.00,104.00,104.00,1,charge_1


In [42]:
prices_adjusted = pd.concat([price_winter, price_summer], axis=0).sort_values(['date', 'hour'])
prices_adjusted

,date,datetime,hour,gb_price_eur,PUN,NORD,CNOR,CSUD,SUD,SARD,SICI,month,flow
0,2022-01-01,2022-01-01 00:00:00,1,89.24,170.28,170.28,170.28,170.28,170.28,170.28,170.28,1,no_flow
1,2022-01-01,2022-01-01 01:00:00,2,89.24,155.72,155.72,155.72,155.72,155.72,155.72,155.72,1,no_flow
2,2022-01-01,2022-01-01 02:00:00,3,108.52,147.09,147.09,147.09,147.09,147.09,147.09,147.09,1,charge_1
3,2022-01-01,2022-01-01 03:00:00,4,57.12,91.00,91.00,91.00,91.00,91.00,91.00,91.00,1,charge_1
4,2022-01-01,2022-01-01 04:00:00,5,83.29,104.00,104.00,104.00,104.00,104.00,104.00,104.00,1,charge_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8754,2022-12-31,2022-12-31 19:00:00,20,191.16,333.00,333.00,333.00,333.00,333.00,333.00,333.00,12,discharge_2
8755,2022-12-31,2022-12-31 20:00:00,21,192.85,310.00,310.00,310.00,310.00,310.00,310.00,310.00,12,no_flow
8756,2022-12-31,2022-12-31 21:00:00,22,194.88,270.00,270.00,270.00,270.00,270.00,270.00,270.00,12,no_flow
8757,2022-12-31,2022-12-31 22:00:00,23,214.28,217.78,217.78,217.78,217.78,217.78,217.78,217.78,12,no_flow


In [43]:
charging_cost = prices_adjusted.loc[(prices_adjusted.flow == 'charge_1') | (prices_adjusted.flow == 'charge_2')].PUN.sum()
discharging_income = prices_adjusted.loc[(prices_adjusted.flow == 'discharge_1') | (prices_adjusted.flow == 'discharge_2')].PUN.sum()
revenue = discharging_income - charging_cost
print(f'2022 4 hour adjusted 2 cycle revenue: {round(revenue, 2)} €/MW.')

2022 4 hour adjusted 2 cycle revenue: 213236.59 €/MW.


In [44]:
charging_1 = prices_adjusted.loc[prices_adjusted.flow == 'charge_1']
discharging_1 = prices_adjusted.loc[prices_adjusted.flow == 'discharge_1']
charging_2 = prices_adjusted.loc[prices_adjusted.flow == 'charge_2']
discharging_2 = prices_adjusted.loc[prices_adjusted.flow == 'discharge_2']

charging_1_max = charging_1.groupby(['date', 'flow'])['PUN'].max().reset_index()
discharging_1_min = discharging_1.groupby(['date', 'flow'])['PUN'].min().reset_index()
charging_2_max = charging_2.groupby(['date', 'flow'])['PUN'].max().reset_index()
discharging_2_min = discharging_2.groupby(['date', 'flow'])['PUN'].min().reset_index()

cycle_1 = pd.merge(charging_1_max, discharging_1_min, on='date', how='inner', suffixes=('_charge_max', '_discharge_min'))
cycle_2 = pd.merge(charging_2_max, discharging_2_min, on='date', how='inner', suffixes=('_charge_max', '_discharge_min'))

cycle_1.head()

,date,flow_charge_max,PUN_charge_max,flow_discharge_min,PUN_discharge_min
0,2022-01-01,charge_1,147.09000,discharge_1,67.99000
1,2022-01-02,charge_1,141.93000,discharge_1,152.59000
2,2022-01-03,charge_1,85.00000,discharge_1,202.03000
3,2022-01-04,charge_1,120.00000,discharge_1,182.64315
4,2022-01-05,charge_1,94.79055,discharge_1,215.98000


In [45]:
(cycle_1.PUN_discharge_min - cycle_1.PUN_charge_max).sum() + (cycle_2.PUN_discharge_min - cycle_2.PUN_charge_max).sum()

23259.30771

In [46]:
def charge_more_than_discharge(charge_max, discharge_min):
    if discharge_min > charge_max: return 0
    else: return 1

cycle_1['bad_profile'] = cycle_1.apply(lambda x: charge_more_than_discharge(x.PUN_charge_max, x.PUN_discharge_min), axis=1)
cycle_2['bad_profile'] = cycle_2.apply(lambda x: charge_more_than_discharge(x.PUN_charge_max, x.PUN_discharge_min), axis=1)
cycle_1.head()

,date,flow_charge_max,PUN_charge_max,flow_discharge_min,PUN_discharge_min,bad_profile
0,2022-01-01,charge_1,147.09000,discharge_1,67.99000,1
1,2022-01-02,charge_1,141.93000,discharge_1,152.59000,0
2,2022-01-03,charge_1,85.00000,discharge_1,202.03000,0
3,2022-01-04,charge_1,120.00000,discharge_1,182.64315,0
4,2022-01-05,charge_1,94.79055,discharge_1,215.98000,0


In [47]:
good_profiles_df = pd.merge(cycle_1, cycle_2, on='date', how='inner', suffixes=('_1', '_2'))
no_good_days = len(good_profiles_df.loc[(good_profiles_df.bad_profile_1 == 0) & (good_profiles_df.bad_profile_2 == 0)])
print(f'Days with a good profile on both cycles: {no_good_days}')

Days with a good profile on both cycles: 234


In [48]:
bad_profiles_df = pd.merge(cycle_1, cycle_2, on='date', how='inner', suffixes=('_1', '_2'))
no_bad_days = len(bad_profiles_df.loc[(bad_profiles_df.bad_profile_1 == 1) & (bad_profiles_df.bad_profile_2 == 1)])
print(f'Days with a bad profile on both cycles: {no_bad_days}')

Days with a bad profile on both cycles: 2


In [49]:
bad_profiles_df.loc[(bad_profiles_df.bad_profile_1 == 1) & (bad_profiles_df.bad_profile_2 == 1)]

,date,flow_charge_max_1,PUN_charge_max_1,flow_discharge_min_1,PUN_discharge_min_1,bad_profile_1,flow_charge_max_2,PUN_charge_max_2,flow_discharge_min_2,PUN_discharge_min_2,bad_profile_2
148,2022-05-29,charge_1,191.32,discharge_1,182.95,1,charge_2,185.96000,discharge_2,179.55811,1
351,2022-12-18,charge_1,260.50,discharge_1,229.50,1,charge_2,262.21039,discharge_2,260.50000,1


In [50]:
cycle_1_count = 365 - cycle_1.bad_profile.sum()
cycle_2_count = 365 - cycle_2.bad_profile.sum()

fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    subplot_titles=['Cycle 1', 'Cycle 2'],
    specs=[[{'secondary_y': True}], [{'secondary_y': True}]]
)

fig.add_trace(go.Scatter(
    x=cycle_1.date,
    y=cycle_1.PUN_charge_max,
    line=dict(color='crimson', width=1.5),
    name='PUN night max'
), secondary_y=True, row=1, col=1)
fig.add_trace(go.Scatter(
    x=cycle_1.date,
    y=cycle_1.PUN_discharge_min,
    line=dict(color='green', width=1.5),
    name='PUN morning min'
), secondary_y=True, row=1, col=1)
fig.add_trace(go.Bar(
    x=cycle_1.date,
    y=cycle_1.bad_profile,
    marker_color='lightgrey',
    name='Bad profile 1'
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=cycle_2.date,
    y=cycle_2.PUN_charge_max,
    line=dict(color='indigo', width=1.5),
    name='PUN afternoon max'
), secondary_y=True, row=2, col=1)
fig.add_trace(go.Scatter(
    x=cycle_2.date,
    y=cycle_2.PUN_discharge_min,
    line=dict(color='mediumvioletred', width=1.5),
    name='PUN evening min'
), secondary_y=True, row=2, col=1)
fig.add_trace(go.Bar(
    x=cycle_2.date,
    y=cycle_2.bad_profile,
    marker_color='lightgrey',
    name='Bad profile 2'
), row=2, col=1)
fig.add_annotation(
    text=f'Cycle 1 good days: <br> {cycle_1_count}/365', 
    font=dict(size=14),
    align='center',
    showarrow=False,
    xref='paper',
    yref='paper',
    x=1.18,
    y=0.5,
    bordercolor='black',
    borderwidth=2,
    )
fig.add_annotation(
    text=f'Cycle 2 good days: <br> {cycle_2_count}/365', 
    font=dict(size=14),
    align='center',
    showarrow=False,
    xref='paper',
    yref='paper',
    x=1.18,
    y=0,
    bordercolor='black',
    borderwidth=2,
    )

fig.update_yaxes(visible=False, secondary_y=False)
fig.update_yaxes(title='€/MWh', secondary_y=True)
fig.update_layout(
    title='Italian MGP 4 hour 2 cycle analysis - winter adjusted',
    template='plotly_white',
    width=1200,
    height=600,
)

fig.write_image('../figures/italy-4hr-profile-winter-adjusted.jpeg', scale=10, engine='orca')
fig.write_html('../figures/italy-4hr-profile-winter-adjusted.html')
fig